## Read pendulum parameters

In [1]:
import yaml
import os
from ament_index_python.packages import get_package_share_directory

pendulum_params_file = os.path.join(
    get_package_share_directory("furuta_pendulum_description"),
    "config",
    "pendulum_parameters.yaml",
)

with open(pendulum_params_file, "r") as file:
    pendulum_params = yaml.safe_load(file)["/**"]["ros__parameters"]

print(pendulum_params)

{'m1': 0.3, 'm2': 0.075, 'l1': 0.15, 'l2': 0.148, 'L1': 0.278, 'L2': 0.3, 'J1': 0.0248, 'J2': 0.00386, 'b1': 0.0001, 'b2': 0.00028, 'L': 0.005, 'R': 7.8, 'Km': 0.09}


In [2]:
g = 9.80665

m1 = pendulum_params["m1"]
m2 = pendulum_params["m2"]

l1 = pendulum_params["l1"]
l2 = pendulum_params["l2"]

L1 = pendulum_params["L1"]
L2 = pendulum_params["L2"]

J1 = pendulum_params["J1"]
J2 = pendulum_params["J2"]

b1 = pendulum_params["b1"]
b2 = pendulum_params["b2"]

J2_hat = J2 + m2 * l2 * l2
J0_hat = J1 + m1 * l1 * l1 + m2 * L1 * L1

## LQR

### Linearized Furuta pendulum

In [3]:
import numpy as np

# Linearized furuta pendulum from
# https://www.hindawi.com/journals/jcse/2011/528341/

denominator = J0_hat * J2_hat - (m2**2.0) * (L1**2.0) * (l2**2.0)

A32 = (g * (m2**2.0) * (l2**2.0) * L1) / denominator
A33 = (-b1 * J2_hat) / denominator
A34 = (-b2 * m2 * l2 * L1) / denominator

A42 = (g * m2 * l2 * J0_hat) / denominator
A43 = (-b1 * m2 * l2 * L1) / denominator
A44 = (-b2 * J0_hat) / denominator

B31 = (J2_hat) / denominator
B41 = (m2 * L1 * l2) / denominator
B32 = (m2 * L1 * l2) / denominator
B42 = (J0_hat) / denominator

A = np.array([[0, 0, 1, 0], [0, 0, 0, 1], [0, A32, A33, A34], [0, A42, A43, A44]])
B = np.array([[0], [0], [B31], [B41]])
# B32 and B42 not used, as I assumed no disturbance (tau2 = 0)

print(A, B)

[[ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.71389430e+00 -2.80773642e-03 -4.40857682e-03]
 [ 0.00000000e+00  2.07426310e+01 -1.57449172e-03 -5.33553802e-02]] [[ 0.        ]
 [ 0.        ]
 [28.07736417]
 [15.7449172 ]]


### Q and R weights

In [4]:
theta1_weight = 0.0
theta2_weight = 10.0
dtheta1_weight = 1.0
dtheta2_weight = 1.0
u_weight = 1.0

Q = np.array(
    [
        [theta1_weight, 0, 0, 0],
        [0, theta2_weight, 0, 0],
        [0, 0, dtheta1_weight, 0],
        [0, 0, 0, dtheta2_weight],
    ]
)
R = np.array([u_weight])

### Calculate K

In [5]:
from control import lqr

K, S, E = lqr(A, B, Q, R)
print(f"{K[0,0]}, {K[0,1]}, {K[0,2]}, {K[0,3]}")

-3.117213617190637e-15, 19.26952361008958, -1.000100004999993, 4.352080693946377


### Save K

In [6]:
import yaml
import os
from ament_index_python.packages import get_package_share_directory

controller_params_file = os.path.join(
    get_package_share_directory("furuta_pendulum_de"),
    "config",
    "lqr_with_swing_up_controller.yaml",
)

with open(controller_params_file, "r") as file:
    controller_params = yaml.safe_load(file)

controller_params["/**"]["ros__parameters"]["K"] = [float(K[0, x]) for x in range(4)]

with open(controller_params_file, "w") as file:
    # default_flow_style=None to get list formatting as [0,1,2] instead of bullet list
    yaml.dump(controller_params, file, default_flow_style=None)
